# Train an SVM or XGBoost model

The focus of this is to train a model in one dataset and try to predict in another from a fifferent cell line. My interest here is to check if I can improve our ability to classify by using the k-mer scores as follows:
 - Create a possitive and a negative set from one ChIP-seq cell line
     - Score the sequence using:
         - E-scores
         - frequency difference
         - a kmer based model generated by training the sequences / using kmer counts of all the 8-mers in each set
     - Use the model generated to classify a given set of sequences
 
 The next question is, wha kind of codes do we need:
     1. A scoring function for all the sequences
     2. A quick way to get the counts of the k-mers / have a look at how some previous challenges have used this approach
     3. A quick way to get the DNA-shape features: This will require choosing a portion of the highest scoring window to use
     4. Test the above features first, and then see if we can expand this to other features
     
     
To be able to achieve all the above, we will need to get an indepth understanding of what we are doing, the algorithm we will use, and how much we will end up doing. 

## Import the useful modules

In [1]:
from multiprocessing import Pool, cpu_count
import subprocess
import pandas as pd
import numpy as np
from math import  exp
import seaborn as sns
import glob
import os

import pybedtools
import pyBigWig
import pysam
pd.set_option('display.max_colwidth', -1)

/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import matplotlib.pyplot as plt

# Main SVM module and grid search function
from sklearn import svm, grid_search

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
#For partitioning the data
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score, KFold

#Libsvm format data loading
from sklearn.datasets import load_svmlight_file

#Accuracy metrics
from sklearn.metrics import accuracy_score, classification_report, auc

# Creating an learning pipeline
from sklearn.pipeline import Pipeline

from sklearn import feature_selection

from sklearn.externals import joblib

#from xgboost import XGBClassifier

import xgboost as xgb

%matplotlib inline

In [3]:
from XGB_TFBSContext import *

# First set the path to essential, but large files

### 1. The DNA Shape files
Downloaded from:

In [4]:
shape_path = "/home/kipkurui/Dream_challenge/DNAShape"

### 2. The human genome

In [5]:
human_genome = "/home/kipkurui/Dream_challenge/annotations"

### 3. Uniformly processed ChIP-seq peaks

In [6]:
chipseq_path = "/home/kipkurui/Project/MARS/Data/ChIP-seq/Downloaded"

In [7]:
#BASE_DIR = "/home/kipkurui/Dream_challenge/DreamChallenge"

### 2. Create kmer dictionaries for the features of interest
We have two option here:
1. Backround noise scalled in a simiklar maner to sticky k-mers 
1. Preferred k-mers max normalized

In [8]:
dn_hg_dict, kmer_name = get_kmer_dict_rev("Data/dn_hg_max_normalized.txt", "test")

dn_hg_dict2, kmer_name = get_kmer_dict_rev("Data/hg_dn_backround_noise_minmax.txt", "test")

### 3. Score the sequences of interest

#### a) K_mer score

### Run all data prepapration steps

In [9]:
def get_feature_df(tf, pos):
    """
    Given a TF and the position of the peak file of interest
    Creat a DataFrame with all the coordinates
    
    This is the main Feature Vector
    """
    peak_files = get_peak_files(tf)

    combined_bed, trim_to = get_combined_bed(peak_files[pos])

    E_score_dict, kmer_name = get_contigmers_dict(get_contigmers(tf)[0],"test")

    ## Calculate all the necessary features
    #E_score_combined = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)

    feature_frame = pd.DataFrame()
    feature_frame["sum_kmer_score"] = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    feature_frame ["max_kmer_score"] = get_kmer_score(combined_bed, max_score_kmer, E_score_dict)
    test_score = get_kmer_score(combined_bed, max_score_kmer_pos, E_score_dict)
    double_deal = test_score.apply(pd.Series)
    feature_frame ["max_kmer_score_pos"] = double_deal[0]
    hits_df = get_hits_df(double_deal, combined_bed)
    feature_frame["dnase"] = apply_get_max_dnase(hits_df)
    feature_frame["phatsCons"] = apply_get_phatscon(hits_df)
    feature_frame["phyloP100way"] = apply_get_phatscon(hits_df, "phyloP100way")
    
    feature_frame["dn_hg_score"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict)
    feature_frame["dn_hg_score2"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict2)
#     feature_frame["pwm_score"] = get_kmer_score(combined_bed, energyscore, get_motif_details(tf))
    feature_frame.reset_index(drop=True, inplace=True)
    pos_tss = get_distance_to_tss(hits_df.head(trim_to))
    neg_tss = get_distance_to_tss(hits_df.tail(trim_to))
    pos_neg_tss = pos_tss.append(neg_tss)
    pos_neg_tss.reset_index(drop=True, inplace=True) 
    feature_frame["tss_dist"] = pos_neg_tss
    for shape in "ProT MGW HelT Roll".split():
        #feature_frame["%s_shape" % shape] = apply_get_shape(hits_df, shape)
        feature_fr = apply_get_full_shape(hits_df).apply(pd.Series)
        feature_fr.columns = get_shape_names(shape)
        feature_frame = feature_frame.T.append(feature_fr.T).T
    return feature_frame, trim_to

Complete Feature list

In [10]:
feat_list = ['max_kmer_score','dnase','kmer_score',"phatsCons",
 'Roll', 'ProT', 'MGW', 'HelT',
 'max_kmer_score_pos','dn_hg_score',
 'dn_hg_score2',"tss_dist", "phyloP100way"]

In [11]:
tf = "Max"

In [12]:
def pop_this(feat):
    try:
        all_feats.pop(all_feats.index(feat))
    except ValueError:
        try:
            for i in range(8):
                all_feats.pop(all_feats.index(feat+"_%i" % i))
        except ValueError:
            pass

### 4. Train a model using the data

In [1]:
# with open("Results/TF_scores_feature_importance_recursive_pop.txt", "a") as tf_scores:
    
#     tf_scores.write("Tf_name\tAll\t")
#     for j in feat_list:
#         tf_scores.write("%s\t" % j)
#     for tf in ["Max"]:
#         tf_scores.write("\n%s\t" % tf)
#         #tf_feats.write("\n%s\t" % tf)
#         print tf
#         pybedtools.cleanup()
        
#         feature_frame, trim_to = get_feature_df(tf, 0)
#         feature_frame_p,trim_to_p =  get_feature_df(tf, -1)
#         y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
#         y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)
        
#         all_feats = list(feature_frame.columns)
        
#         #All
#         my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
#         testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
#         y_pred = my_model.predict(testdmat)
#         tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
#         for feats in feat_list:
#             all_feats = list(feature_frame.columns)
#             pop_this(feats)
#             my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
            
#             testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)

#             y_pred = my_model.predict(testdmat)

#             tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

In [ ]:
def get_feature_df_master(tf, pos, run_type="base_line"):
    """
    Given a TF and the position of the peak file of interest
    Creat a DataFrame with all the coordinates
    
    This is the main Feature Vector
    """
    peak_files = get_peak_files(tf)

    combined_bed, trim_to = get_combined_bed(peak_files[pos])

    E_score_dict, kmer_name = get_contigmers_dict(get_contigmers(tf)[0],"test")

    ## Calculate all the necessary features
    #E_score_combined = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    
    #Baseline model

    feature_frame = pd.DataFrame()

    test_score = get_kmer_score(combined_bed, max_score_kmer, E_score_dict)
    double_deal = test_score.apply(pd.Series)
    hits_df = get_hits_df(double_deal, combined_bed)
    
    feature_frame ["max_kmer_score"] = double_deal[0]
    
    feature_frame["dnase"] = apply_get_max_dnase(hits_df)
    
    if kmers:
        feature_frame ["max_kmer_score_pos"] = get_kmer_score(combined_bed, max_score_kmer_pos, E_score_dict)
        feature_frame["sum_kmer_score"] = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    if conservation:
        feature_frame["phatsCons"] = apply_get_phatscon(hits_df)
        feature_frame["phyloP100way"] = apply_get_phatscon(hits_df, "phyloP100way")
    
    feature_frame["dn_hg_score"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict)
    feature_frame["dn_hg_score2"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict2)

    feature_frame.reset_index(drop=True, inplace=True)
    
    #TSS
    pos_tss = get_distance_to_tss(hits_df.head(trim_to))
    neg_tss = get_distance_to_tss(hits_df.tail(trim_to))
    pos_neg_tss = pos_tss.append(neg_tss)
    pos_neg_tss.reset_index(drop=True, inplace=True) 
    feature_frame["tss_dist"] = pos_neg_tss
    
    #Shape features
    for shape in "ProT MGW HelT Roll".split():
        #feature_frame["%s_shape" % shape] = apply_get_shape(hits_df, shape)
        feature_fr = apply_get_full_shape(hits_df).apply(pd.Series)
        feature_fr.columns = get_shape_names(shape)
        feature_frame = feature_frame.T.append(feature_fr.T).T
        
    #The PWM
    #feature_frame["pwm_score"] = get_kmer_score(combined_bed, energyscore, get_motif_details(tf))
    return feature_frame, trim_to

In [ ]:
with open("TF_scores_feature_importance_recursive_all.txt", "a") as tf_scores:
    
    tf_scores.write("Tf_name\tAll\t")
    for j in feat_list:
        tf_scores.write("%s\t" % j)
    for tf in repeat_tfs:
        tf_scores.write("\n%s\t" % tf)
        #tf_feats.write("\n%s\t" % tf)
        print tf

        feature_frame, trim_to = get_feature_best(tf, 0)
        feature_frame_p,trim_to_p =  get_feature_best(tf, -1)
        y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
        y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)
        
        all_feats = list(feature_frame.columns)
        
        #All
        my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
        testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
        y_pred = my_model.predict(testdmat)
        tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
        for feats in feat_list:
            all_feats = list(feature_frame.columns)
            pop_this(feats)
            my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
            
            testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

In [19]:
#sns.heatmap(feature_frame_p.corr(), annot=True)

In [17]:
pbm_chip = []
pbmchip2name = {}
with open("Data/Pbm_Chip_details.txt") as pbmnchip:
    for line in pbmnchip:
        if line.startswith('Tf_id'):
            continue
        else:
            pbm_chip.append(line.split()[0])
            pbmchip2name[line.split()[1]] = line.split()[0]

## List of TFs affected by Sticky k-mers

In [8]:
sticky_tfs = pd.read_table("Data/names.txt", header=None)
tf_list = []
for tf in sticky_tfs[0]:
    chip_list = glob.glob("/home/kipkurui/Project/MARS/Data/ChIP-seq/Derived/Posneg/%s/*" % tf.capitalize())
    if len(chip_list) > 0:
        tf_list.append(tf)

In [10]:
tf_list = ['Foxa2',
 'Gata3',
 'Max',
 'Tcf3',
 'Tcf7l2',
 'Irf3',
 'Irf4',
 'Hnf4a',
 'Nr2f2',
 'Rxra',
 'Egr1',
 'Sp4']

### Get a list of Tfs available in PBM and ChIP with more than two peaks

In [11]:
tf_pbm = pd.read_table("Data/Pbm_Chip_details.txt")

in_both = tf_pbm[(tf_pbm["Chip_name"] >0) == (tf_pbm["Pbm_name"] >0)]
chip_name = tf_pbm[(tf_pbm["Chip_name"] >0)]["Chip_name"]
chip_name = chip_name.sort_values()

in_both_new = []
for tf in chip_name:
    if (len(get_contigmers(tf)) > 0) & (len(get_peak_files(tf)) > 1):
        #print get_contigmers(tf)
        in_both_new.append(tf)
#Remove Taf1 -- Wrongly picked above
in_both_new.pop(in_both_new.index("Taf1"))

NameError: name 'get_contigmers' is not defined

In [19]:
in_both_new

['Ap2',
 'Arid3a',
 'Egr1',
 'Elk1',
 'Elk4',
 'Ets1',
 'Gabp',
 'Gata3',
 'Gr',
 'Hnf4a',
 'Irf3',
 'Jund',
 'Mafk',
 'Max',
 'Pou2f2',
 'Rxra',
 'Sp1',
 'Srf',
 'Tbp',
 'Tcf7l2']

In [23]:
in_both_new2 = ['Ap2',
 'Arid3a',
 'Egr1',
 'Elk1',
 'Elk4',
 'Ets1',
 'Gabp',
 'Gata3',
 'Gr',
 'Hnf4a',
 'Irf3',
 'Jund',
 'Mafk',
 'Max',
 'Pou2f2',
 'Rxra',
 'Sp1',
 'Srf',
 'Tbp',
 'Tcf7l2']

### Same as above, but older

Kept for reference purposes only, unless we find some use later

In [21]:
# to_dict = in_both.set_index("Chip_name").drop("Tf_id", 1)
# pbm2chip = to_dict.to_dict()['Pbm_name']

# my_new_list = []
# for tf in pbm2chip:
#     chip_list = glob.glob("/home/kipkurui/Project/MARS/Data/ChIP-seq/Downloaded/*%s*" % pbm2chip[tf].capitalize())
#     if len(chip_list) > 1:
#         my_new_list.append(tf)

In [ ]:
# my_new_list = ['Srf','Hnf4a',
#  'Arid3a',
#  'Tbp',
#  'Elk1',
#  'Elk4',
#  'Gata3',
#  'Irf3',
#   'Tcf7l2',
#  'Pou2f2',
#  'Egr1',
#  'Rxra',
#  'Ets1',
#  'Mafk',
#  'Max']

In [31]:
best = ['max_kmer_score',"phatsCons",'dn_hg_score','dnase', "tss_dist"]

In [111]:
feat_list = [
 'max_kmer_score',"phatsCons",
 'dn_hg_score2',
 'dnase', "tss_dist", "phyloP100way"]

In [ ]:
# for tf in in_both_new:
#     print tf
#     print pbmchip2name[tf]
#     get_motif_details(tf)

In [55]:
overal_list = []
for feat in feat_list:
    feats = feat_list[:]
    feats.pop(feats.index(feat))
    overal_list.append(feats[:])
for feat1 in feat_list:
    new_l = [feat1]
    for feat in feat_list:
        if not feat in new_l:
            new_l.append(feat)
            new_l.sort()
            add_in = new_l[:]
            if add_in not in overal_list:
                overal_list.append(add_in)

In [ ]:
with open("feature_details_importance", "w") as feats:   
    for j, yu in enumerate(overal_list):
        #print feat_list[j]+"_"+str(j)
        feats.write("AUC_%i\t %s\n" % (j, '|'.join(yu)))

In [138]:
#all_feats = list(feature_frame2.columns)

In [111]:
# def pop_this(remove):
#     for feat in list(feature_frame2.columns):
#         if remove in feat:
#             all_feats.pop(all_feats.index(feat))

In [26]:
#pop_this("dnase")

In [27]:
#feature_frame2, trim_to = get_feature_df(tf, 0)

In [24]:
def get_feature_best6(tf, pos):
    peak_files = get_peak_files(tf)

    combined_bed, trim_to = get_combined_bed(peak_files[pos])

    E_score_dict, kmer_name = get_contigmers_dict(get_contigmers(tf)[0],"test")

    ## Calculate all the necessary features
    #E_score_combined = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)

    feature_frame = pd.DataFrame()
#     feature_frame["sum_kmer_score"] = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    #feature_frame["dnase"] = apply_get_max_dnase(combined_bed)
    feature_frame ["max_kmer_score"] = get_kmer_score(combined_bed, max_score_kmer, E_score_dict)
    test_score = get_kmer_score(combined_bed, max_score_kmer_pos, E_score_dict)
    double_deal = test_score.apply(pd.Series)
#     feature_frame ["max_kmer_score_pos"] = double_deal[0]
    hits_df = get_hits_df(double_deal, combined_bed)
    feature_frame["dnase"] = apply_get_max_dnase(hits_df)
#     feature_frame["phatsCons"] = apply_get_phatscon(hits_df)
#     feature_frame["phyloP100way"] = apply_get_phatscon(hits_df, "phyloP100way")
    
#     feature_frame["dn_hg_score"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict)
#     feature_frame["dn_hg_score2"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict2)
#     feature_frame["pwm_score"] = get_kmer_score(combined_bed, energyscore, get_motif_details(tf))
    feature_frame.reset_index(drop=True, inplace=True)
    pos_tss = get_distance_to_tss(hits_df.head(trim_to))
    neg_tss = get_distance_to_tss(hits_df.tail(trim_to))
    pos_neg_tss = pos_tss.append(neg_tss)
    pos_neg_tss.reset_index(drop=True, inplace=True) 
    feature_frame["tss_dist"] = pos_neg_tss
#     for shape in "ProT MGW HelT Roll".split():
#         #feature_frame["%s_shape" % shape] = apply_get_shape(hits_df, shape)
#         feature_fr = apply_get_full_shape(hits_df).apply(pd.Series)
#         feature_fr.columns = get_shape_names(shape)
#         feature_frame = feature_frame.T.append(feature_fr.T).T
    return feature_frame, trim_to

## Feature importance by eliminating one, sequentially

The shape features will have to be eliminated together as a group.This is an attempt to be clear on the contribution to teh accuracy by each of the features. 

Next, I need to 

In [109]:
repeat_tfs = ["Gr","Tbp"]

In [116]:
with open("TF_scores_feature_importance_recursive_all.txt", "a") as tf_scores:
    
    #tf_scores.write("Tf_name\tAll\t")
    #for j in feat_list:
        #tf_scores.write("%s\t" % j)
    for tf in repeat_tfs:
        tf_scores.write("\n%s\t" % tf)
        #tf_feats.write("\n%s\t" % tf)
        print tf

        feature_frame, trim_to = get_feature_best(tf, 0)
        feature_frame_p,trim_to_p =  get_feature_best(tf, -1)
        y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
        y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)
        
        all_feats = list(feature_frame.columns)
        
        #All
        my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
        testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
        y_pred = my_model.predict(testdmat)
        tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
        for feats in feat_list:
            all_feats = list(feature_frame.columns)
            pop_this(feats)
            my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
            
            testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

Gr


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tbp


## Feature importance by recursive addition

In [112]:
with open("TF_scores_feature_importance_recursive_all.txt", "a") as tf_scores:
    tf_scores.write("Tf_name\tAll\t")
    feat_list = ['kmer_score',"phatsCons",
 'Roll', 'ProT', 'MGW', 'HelT',
 'max_kmer_score_pos','dn_hg_score',
 'dn_hg_score2',"tss_dist", "phyloP100way"]
    for j in feat_list:
        tf_scores.write("%s\t" % j)
    for tf in repeat_tfs: #in_both_new:
        tf_scores.write("\n%s\t" % tf)
#         #tf_feats.write("\n%s\t" % tf)
        print tf

        feature_frame, trim_to = get_feature_df(tf, 0)
        feature_frame_p,trim_to_p =  get_feature_df(tf, -1)
        y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
        y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)
        
        all_feats = list(feature_frame.columns)
        
#         #All
        my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
        testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
        y_pred = my_model.predict(testdmat)
        tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
        all_feats = list(feature_frame.columns)
        
        feat_list = ['kmer_score',"phatsCons",
 'Roll', 'ProT', 'MGW', 'HelT',
 'max_kmer_score_pos','dn_hg_score',
 'dn_hg_score2',"tss_dist", "phyloP100way"]
        loop_this = feat_list[:]
        for i,j in enumerate(loop_this):
            all_feats = list(feature_frame.columns)
            feat_list = ['kmer_score',"phatsCons",
             'Roll', 'ProT', 'MGW', 'HelT',
             'max_kmer_score_pos','dn_hg_score',
             'dn_hg_score2',"tss_dist", "phyloP100way"]
            #print i,j
            feat_list.pop(i)

            for i in feat_list:
                pop_this(i)
#             print all_feats
#         for feats in feat_list:
#             all_feats = list(feature_frame.columns)
#             pop_this(feats)
            my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
            
            testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

Gr


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tbp


In [143]:
pybedtools.cleanup()

From the above, we cal confidently deduce that the most informative feature as dnase and tss...however, for other features, we lose information on the quality of the model since k-mer scores are will not be confidently measured. Therefore, we eliminate the poorly performing features and then introduce. 

## Contribution of the DNA shape to the baseline model

Here, we will have a complete feature with:
* The max k-mer score
* The DNase score
* The Each of the shape features

So the Idea is to start with a complete model, then one with a  variation of each of the shape features. 

The difficulty with these is that the model does not consider the order of the features, rather, it starts with the first ones and moves along with the rest. So, generally, the feature presented first seems to have high contribution to the tree decisions. 

Here we want to observe the contribution, rather than how much a dip adding the feature causes. Then decide on the contribution of each of the features. 

In [25]:
def get_feature_df_shape(tf, pos):   
    peak_files = get_peak_files(tf)

    combined_bed, trim_to = get_combined_bed(peak_files[pos])

    E_score_dict, kmer_name = get_contigmers_dict(get_contigmers(tf)[0],"test")

    feature_frame = pd.DataFrame()
#     feature_frame["sum_kmer_score"] = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    #feature_frame["dnase"] = apply_get_max_dnase(combined_bed)
    feature_frame ["max_kmer_score"] = get_kmer_score(combined_bed, max_score_kmer, E_score_dict)
    test_score = get_kmer_score(combined_bed, max_score_kmer_pos, E_score_dict)
    double_deal = test_score.apply(pd.Series)
#     feature_frame ["max_kmer_score_pos"] = double_deal[0]
    hits_df = get_hits_df(double_deal, combined_bed)
    feature_frame["dnase"] = apply_get_max_dnase(hits_df)
#     feature_frame["phatsCons"] = apply_get_phatscon(hits_df)
#     feature_frame["phyloP100way"] = apply_get_phatscon(hits_df, "phyloP100way")
    
#     feature_frame["dn_hg_score"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict)
#     feature_frame["dn_hg_score2"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict2)
#     feature_frame["pwm_score"] = get_kmer_score(combined_bed, energyscore, get_motif_details(tf))
    feature_frame.reset_index(drop=True, inplace=True)
#     pos_tss = get_distance_to_tss(hits_df.head(trim_to))
#     neg_tss = get_distance_to_tss(hits_df.tail(trim_to))
#     pos_neg_tss = pos_tss.append(neg_tss)
#     pos_neg_tss.reset_index(drop=True, inplace=True) 
#     feature_frame["tss_dist"] = pos_neg_tss
    for shape in "ProT MGW HelT Roll".split():
        #feature_frame["%s_shape" % shape] = apply_get_shape(hits_df, shape)
        feature_fr = apply_get_full_shape(hits_df).apply(pd.Series)
        feature_fr.columns = get_shape_names(shape)
        feature_frame = feature_frame.T.append(feature_fr.T).T
    return feature_frame, trim_to

In [123]:
shapes = [ 'Roll', 'ProT', 'MGW', 'HelT']

In [124]:
with open("TF_scores_feature_importance_recursive_shape3.txt", "a") as tf_scores:
    tf_scores.write("Tf_name\tAll\tNone\t")
    for j in [ 'Roll', 'ProT', 'MGW', 'HelT']:
        tf_scores.write("%s\t" % j)
    for tf in repeat_tfs:#in_both_new[17:]:
        shapes = [ 'Roll', 'ProT', 'MGW', 'HelT']
        tf_scores.write("\n%s\t" % tf)
        #tf_feats.write("\n%s\t" % tf)
        print tf

        feature_frame, trim_to = get_feature_df_shape(tf, 0)
        feature_frame_p,trim_to_p =  get_feature_df_shape(tf, -1)
        y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
        y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)
        
        all_feats = list(feature_frame.columns)
        
        #All
        my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
        testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
        y_pred = my_model.predict(testdmat)
        tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
        for i in shapes:
            pop_this(i)
        my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
        testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
        y_pred = my_model.predict(testdmat)
        tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
        all_feats = list(feature_frame.columns)
        for i,j in enumerate([ 'Roll', 'ProT', 'MGW', 'HelT']):
            all_feats = list(feature_frame.columns)
            shapes = [ 'Roll', 'ProT', 'MGW', 'HelT']
            #print i,j
            shapes.pop(i)

            for i in shapes:
                pop_this(i)
    
#         for feats in feat_list:
#             all_feats = list(feature_frame.columns)
#             pop_this(feats)
            my_model = train_xgboost(feature_frame[all_feats], y_train, tf)

            testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

Gr


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tbp


## Test the effect of the preferred and noise k-mer scores as an additional feature

Question here is to determine if it does contribute to the predictive ability of the model. Here we can extract the information from the feature contribution based on how much loss it causes. 

Here, we also want to use the baseline model comprosing of the DNase and kmer scores...that is after determining the best scoring function we just settle on that for any subsequent computation. The level of correlation of the various features is also informative in terms of how much more value they can add to the quality of the model. 

With this, the best option, is to test the performance of the baseline with k-mer model and with Dnase data

In [129]:
noise_features = [["dnase", "max_kmer_score","dn_hg_score","dn_hg_score2"],["dnase", "max_kmer_score"],["dnase", "max_kmer_score","dn_hg_score2"], ["dnase", "max_kmer_score","dn_hg_score"]]

In [25]:
def get_feature_noise(tf, pos):
    peak_files = get_peak_files(tf)

    combined_bed, trim_to = get_combined_bed(peak_files[pos])

    E_score_dict, kmer_name = get_contigmers_dict(get_contigmers(tf)[0],"test")

    feature_frame = pd.DataFrame()
#     feature_frame["sum_kmer_score"] = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    #feature_frame["dnase"] = apply_get_max_dnase(combined_bed)
    feature_frame ["max_kmer_score"] = get_kmer_score(combined_bed, max_score_kmer, E_score_dict)
    test_score = get_kmer_score(combined_bed, max_score_kmer_pos, E_score_dict)
    double_deal = test_score.apply(pd.Series)
#     feature_frame ["max_kmer_score_pos"] = double_deal[0]
    hits_df = get_hits_df(double_deal, combined_bed)
    feature_frame["dnase"] = apply_get_max_dnase(hits_df)
#     feature_frame["phatsCons"] = apply_get_phatscon(hits_df)
#     feature_frame["phyloP100way"] = apply_get_phatscon(hits_df, "phyloP100way")
    
    feature_frame["dn_hg_score"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict)
    feature_frame["dn_hg_score2"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict2)
#     feature_frame["pwm_score"] = get_kmer_score(combined_bed, energyscore, get_motif_details(tf))
    feature_frame.reset_index(drop=True, inplace=True)
#     pos_tss = get_distance_to_tss(hits_df.head(trim_to))
#     neg_tss = get_distance_to_tss(hits_df.tail(trim_to))
#     pos_neg_tss = pos_tss.append(neg_tss)
#     pos_neg_tss.reset_index(drop=True, inplace=True) 
#     feature_frame["tss_dist"] = pos_neg_tss
#     for shape in "ProT MGW HelT Roll".split():
#         #feature_frame["%s_shape" % shape] = apply_get_shape(hits_df, shape)
#         feature_fr = apply_get_full_shape(hits_df).apply(pd.Series)
#         feature_fr.columns = get_shape_names(shape)
#         feature_frame = feature_frame.T.append(feature_fr.T).T
    return feature_frame, trim_to

In [80]:
in_both_new[16:]

['Sp1', 'Srf', 'Tbp', 'Tcf7l2']

In [131]:
with open("TF_scores_feature_importance_recursive_noise.txt", "a") as tf_scores:
    #tf_scores.write("Tf_name\tAll\tNone\tNoise\tPreferred\t")
    #for j in feat_list:
        #tf_scores.write("%s\t" % j)
    for tf in repeat_tfs:
        tf_scores.write("\n%s\t" % tf)
        #tf_feats.write("\n%s\t" % tf)
        print tf

        feature_frame, trim_to = get_feature_noise(tf, 0)
        #for pos in range(1,len(get_peak_files(tf)))
        feature_frame_p,trim_to_p =  get_feature_noise(tf, -1)
        y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
        y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)
        
        all_feats = list(feature_frame.columns)
        
        #All
#         my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
#         testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
#         y_pred = my_model.predict(testdmat)
#         tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
        for feats in noise_features:
            #all_feats = list(feature_frame.columns)
            #pop_this(feats)
            my_model = train_xgboost(feature_frame[feats], y_train, tf)
            
            testdmat = xgb.DMatrix(feature_frame_p[feats], y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

Gr


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tbp


## Test how well the models can be generalizable

To do this, I need to identify those TFS that have data in more than three cell lines, then use that infromation to test how a model trained in a rottatting manner can be used top predict binding in teh other cell lines and, what is the accuracy. Although, in a way, our current implementation where we train in one and test in another is okay, we need to check and see if there exists flactuations in performance depending on the training cell line. 

Given a TF with more than one cell line, we get the name and then test prediction ability of a model from one cell line in predicting performance in another cell line.
* Start with single cell line, and if we do observe some irregularities, then --
* Create a  model from each of the cell lines testing the perfomance in the other cell lines. 
* Determine how well we can generalize our predictions

In [26]:
over_3 = []
for tf in in_both_new:
    #print tf
    peak_files = get_peak_files(tf)
    if (len(peak_files) > 3) & (len(peak_files) < 10):
        over_3.append(tf)
        print tf
    #print get_celltype(peak_files)

Gabp
Gata3
Gr
Jund
Mafk
Max
Sp1
Srf
Tbp
Tcf7l2


In [27]:
def get_celltype(peak_files):
    cell_types = []
    for i in range(len(peak_files)):
        cell_types.append(peak_files[i].split("/")[-1].split("Tfbs")[-1].split("UniPk")[0])
    return cell_types

In [28]:
def get_feature_cell_type(tf, pos):
    peak_files = get_peak_files(tf)

    combined_bed, trim_to = get_combined_bed(peak_files[pos])

    E_score_dict, kmer_name = get_contigmers_dict(get_contigmers(tf)[0],"test")

    feature_frame = pd.DataFrame()
#     feature_frame["sum_kmer_score"] = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    #feature_frame["dnase"] = apply_get_max_dnase(combined_bed)
    feature_frame ["max_kmer_score"] = get_kmer_score(combined_bed, max_score_kmer, E_score_dict)
    test_score = get_kmer_score(combined_bed, max_score_kmer_pos, E_score_dict)
    double_deal = test_score.apply(pd.Series)
#     feature_frame ["max_kmer_score_pos"] = double_deal[0]
    hits_df = get_hits_df(double_deal, combined_bed)
    feature_frame["dnase"] = apply_get_max_dnase(hits_df)
    feature_frame["phatsCons"] = apply_get_phatscon(hits_df)
    feature_frame["phyloP100way"] = apply_get_phatscon(hits_df, "phyloP100way")
    
    feature_frame["dn_hg_score"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict)
    feature_frame["dn_hg_score2"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict2)
#     feature_frame["pwm_score"] = get_kmer_score(combined_bed, energyscore, get_motif_details(tf))
    feature_frame.reset_index(drop=True, inplace=True)
    pos_tss = get_distance_to_tss(hits_df.head(trim_to))
    neg_tss = get_distance_to_tss(hits_df.tail(trim_to))
    pos_neg_tss = pos_tss.append(neg_tss)
    pos_neg_tss.reset_index(drop=True, inplace=True) 
    feature_frame["tss_dist"] = pos_neg_tss
    for shape in "ProT MGW HelT Roll".split():
        #feature_frame["%s_shape" % shape] = apply_get_shape(hits_df, shape)
        feature_fr = apply_get_full_shape(hits_df).apply(pd.Series)
        feature_fr.columns = get_shape_names(shape)
        feature_frame = feature_frame.T.append(feature_fr.T).T
    return feature_frame, trim_to

In [138]:
with open("TF_scores_cell_type_specificity_1.txt", "a") as tf_scores:
    #tf_scores.write("Tf_name\tAll\tNone\tNoise\tPreferred\t")
    #for j in feat_list:
        #tf_scores.write("%s\t" % j)
    for tf in reapeat_tfs:
        tf_scores.write("\n%s\t" % tf)
        #tf_feats.write("\n%s\t" % tf)
        print tf

        feature_frame, trim_to = get_feature_cell_type(tf, -1)
        y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
        my_model = train_xgboost(feature_frame, y_train, tf)
        for pos in range(0,len(get_peak_files(tf))-1):
            feature_frame_p,trim_to_p =  get_feature_cell_type(tf, pos)

            y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)

            all_feats = list(feature_frame.columns)

            testdmat = xgb.DMatrix(feature_frame_p, y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

Gr


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tbp


In [140]:
for i in range(len(get_peak_files(tf))):
    print i

0
1
2
3
4


In [144]:
over_3

['Gabp', 'Gata3', 'Gr', 'Jund', 'Mafk', 'Max', 'Sp1', 'Srf', 'Tbp', 'Tcf7l2']

## Test the different training cell types

In [32]:
over_3 = ['Gabp', 'Gata3', 'Jund', 'Mafk', 'Max', 'Sp1', 'Srf', 'Tbp', 'Tcf7l2']

In [35]:
pybedtools.cleanup()

In [29]:
over_3 = ['Max']

In [30]:
with open("TF_scores_cell_type_specificity_recursive.txt", "a") as tf_scores:
    #tf_scores.write("Tf_name\tAll\tNone\tNoise\tPreferred\t")
    #for j in feat_list:
        #tf_scores.write("%s\t" % j)
    for tf in over_3:
        tf_scores.write("\n%s\n" % tf)
        #tf_feats.write("\n%s\t" % tf)
        print tf
        for train in range(len(get_peak_files(tf))):
            pybedtools.cleanup()
            tf_scores.write("%s\t" % get_celltype(get_peak_files(tf))[train])
            feature_frame, trim_to = get_feature_cell_type(tf, train)
            y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
            my_model = train_xgboost(feature_frame, y_train, tf)
            for pos in range(len(get_peak_files(tf))):
                feature_frame_p,trim_to_p =  get_feature_cell_type(tf, pos)

                y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)

                all_feats = list(feature_frame.columns)

                testdmat = xgb.DMatrix(feature_frame_p, y_test)

                y_pred = my_model.predict(testdmat)

                tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

Max


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Test for conservation data contribution

Using the same idea, we need to test the various forms of acquiring the consevation scores and their effect on the perfomance of the model. Here are to test the following:
* Phastcons hit
* phastcons whole site
* Phylo hit
* phylo whole site

With each of these, we use the baseline model already defined and tested. 


In [132]:
def get_feature_conservation(tf, pos):
    peak_files = get_peak_files(tf)

    combined_bed, trim_to = get_combined_bed(peak_files[pos])

    E_score_dict, kmer_name = get_contigmers_dict(get_contigmers(tf)[0],"test")

    feature_frame = pd.DataFrame()
#     feature_frame["sum_kmer_score"] = get_kmer_score(combined_bed, sum_kmer_score, E_score_dict)
    #feature_frame["dnase"] = apply_get_max_dnase(combined_bed)
    feature_frame ["max_kmer_score"] = get_kmer_score(combined_bed, max_score_kmer, E_score_dict)
    test_score = get_kmer_score(combined_bed, max_score_kmer_pos, E_score_dict)
    double_deal = test_score.apply(pd.Series)
#     feature_frame ["max_kmer_score_pos"] = double_deal[0]
    hits_df = get_hits_df(double_deal, combined_bed)
    feature_frame["dnase"] = apply_get_max_dnase(hits_df)
    feature_frame["phatsCons"] = apply_get_phatscon(hits_df)
    feature_frame["phyloP100way"] = apply_get_phatscon(hits_df, "phyloP100way")
    
    feature_frame["phatsCons_whole"] = apply_get_phatscon(combined_bed)
    feature_frame["phyloP100way_whole"] = apply_get_phatscon(combined_bed, "phyloP100way")
    
#     feature_frame["dn_hg_score"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict)
#     feature_frame["dn_hg_score2"] = get_kmer_score(combined_bed, max_score_kmer, dn_hg_dict2)
#     feature_frame["pwm_score"] = get_kmer_score(combined_bed, energyscore, get_motif_details(tf))
    feature_frame.reset_index(drop=True, inplace=True)
#     pos_tss = get_distance_to_tss(hits_df.head(trim_to))
#     neg_tss = get_distance_to_tss(hits_df.tail(trim_to))
#     pos_neg_tss = pos_tss.append(neg_tss)
#     pos_neg_tss.reset_index(drop=True, inplace=True) 
#     feature_frame["tss_dist"] = pos_neg_tss
#     for shape in "ProT MGW HelT Roll".split():
#         #feature_frame["%s_shape" % shape] = apply_get_shape(hits_df, shape)
#         feature_fr = apply_get_full_shape(hits_df).apply(pd.Series)
#         feature_fr.columns = get_shape_names(shape)
#         feature_frame = feature_frame.T.append(feature_fr.T).T
    return feature_frame, trim_to

In [133]:
conservation_features = [["dnase", "max_kmer_score","phatsCons","phyloP100way", "phatsCons_whole","phyloP100way_whole"],
                  ["dnase", "max_kmer_score"],
                  ["dnase", "max_kmer_score","phyloP100way", "phatsCons_whole","phyloP100way_whole"],
                  ["dnase", "max_kmer_score","phatsCons", "phatsCons_whole","phyloP100way_whole"],
                 ["dnase", "max_kmer_score","phatsCons","phyloP100way","phyloP100way_whole"],
                 ["dnase", "max_kmer_score","phatsCons","phyloP100way", "phatsCons_whole"]]

In [134]:
with open("TF_scores_feature_importance_recursive_conservation.txt", "a") as tf_scores:
    #tf_scores.write("Tf_name\tAll\tNone\tPhats_hit\tPhylo_hit\tPhats_wh\tPhylo_wh\t")
    #for j in feat_list:
        #tf_scores.write("%s\t" % j)
    for tf in reapeat_tfs:
        tf_scores.write("\n%s\t" % tf)
        #tf_feats.write("\n%s\t" % tf)
        print tf

        feature_frame, trim_to = get_feature_conservation(tf, 0)
        feature_frame_p,trim_to_p =  get_feature_conservation(tf, -1)
        y_train = np.concatenate((np.ones(trim_to), np.zeros(trim_to)), axis=0)
        y_test = np.concatenate((np.ones(trim_to_p), np.zeros(trim_to_p)), axis=0)
        
        all_feats = list(feature_frame.columns)
        
        #All
#         my_model = train_xgboost(feature_frame[all_feats], y_train, tf)
#         testdmat = xgb.DMatrix(feature_frame_p[all_feats], y_test)
#         y_pred = my_model.predict(testdmat)
#         tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))
        
        for feats in conservation_features:
            #all_feats = list(feature_frame.columns)
            #pop_this(feats)
            my_model = train_xgboost(feature_frame[feats], y_train, tf)
            
            testdmat = xgb.DMatrix(feature_frame_p[feats], y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t" % (roc_auc_score(y_test, y_pred)))

Gr


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tbp


## Quickly compare the performance of the k-mer and the PWM models both derived from PBM data

Here, the focus is to test the best scoring function for k-mers and the PWM derived from the same data. The intention is to determine if there exists any performance difference. 

For the pure k-mers, I had previously tested an option of scoring using k-mer above a given threshold. The benefit of this is that it will not be confounded by the poorly scoring k-mers. However, for a given sequence, teh number of k-mers above the trheshold will shift the scores in their favour, though it can be argues that this will still help pick up sequences with high scoring k-mers.

The initial testing of this did not reveal this benefit...but it will not hurt to have a more comprehensive test, as well as for the thresholds (common ones are: 0.25 or 0.35)

In [69]:
with open("TF_scores_best_shape_everything.txt", "a") as tf_scores:
    with open("TF_feats_best_shape_everything.txt", "a") as tf_feats:
        
        tf_scores.write("Tf_name\tAUC\tAUPRC\t")
#         for i in feat_list:
#             tf_scores.write(i+"\t")
#             tf_feats.write(i+"\t")
        for tf in in_both_new:
            tf_scores.write("\n%s\t" % tf)
            tf_feats.write("\n%s\t" % tf)
            print tf

            feature_frame, trim_to = get_feature_df(tf, 0)
            #feature_frame = shuffle_df_columns(feature_frame)
            neg_size = trim_to
            pos_size = trim_to
            y_train = np.concatenate((np.ones(pos_size), np.zeros(neg_size)), axis=0)

            my_model = train_xgboost(feature_frame, y_train, tf)

            feature_frame_p,trim_to_p =  get_feature_df(tf, -1)
            
            #shuffling the df columns to test feature importance
            #feature_frame_p = shuffle_df_columns(feature_frame_p)


            neg_size = trim_to_p #len(pos_bed_p)
            pos_size = trim_to_p #len(neg_bed_p)

            y_test = np.concatenate((np.ones(pos_size), np.zeros(neg_size)), axis=0)

            testdmat = xgb.DMatrix(feature_frame_p, y_test)

            y_pred = my_model.predict(testdmat)

            tf_scores.write("%s\t%s\t" % (roc_auc_score(y_test, y_pred),calc_auPRC(y_test, y_pred)))

            #print calc_auPRC(y_test, y_pred)
            print "AUPRC", roc_auc_score(y_test, y_pred)
#             importances = my_model.get_fscore()
#             for feat in feat_list:
#                 pwm_score = feature_frame_p[feat]
#                 pwm_score = np.array(pwm_score)
#                 tf_scores.write("%s\t" % roc_auc_score(y_test, pwm_score))
#                 tf_feats.write("%s\t" % importances[feat])
                
            

                

Ap2


/home/kipkurui/anaconda2/envs/dream_challenge/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AUPRC 0.977606933355
Arid3a
AUPRC 0.880469079034
Egr1
AUPRC 0.910383578774
Elk1
AUPRC 0.97223757191
Elk4
AUPRC 0.960571050507
Ets1
AUPRC 0.943579325179
Gabp
AUPRC 0.952161968975
Gata3
AUPRC 0.907285569572
Gr
AUPRC 0.797471857658
Hnf4a
AUPRC 0.979244390734
Irf3
AUPRC 0.93453141081
Jund
AUPRC 0.911029543485
Mafk
AUPRC 0.963480840785
Max
AUPRC 0.928093808582
Pou2f2
AUPRC 0.945298583355
Rxra
AUPRC 0.711409542972
Sp1
AUPRC 0.908936667799
Srf
AUPRC 0.964137669576
Tbp
AUPRC 0.681431935356
Tcf7l2
AUPRC 0.863327983539


In [33]:
pybedtools.cleanup()

In [55]:
def shuffle_df_columns(df):
    "This really doesn't add much value"
    import random
    asd = list(df.columns)
    random.seed(12456)
    random.shuffle(asd)
    return df[asd]

In [ ]:
0.967028447896
Arid3a
0.852579262049
Egr1
0.911651140092
Elk1
0.97114564576
Elk4
0.962128973571
Ets1
0.932960986153
Gabp
0.947173738305
Gata3
0.887761995812
Gr
0.743983361148
Hnf4a
0.965655318165
Irf3
0.927597905978
Jund
0.91214940445
Mafk
0.909838090472
Max
0.915759746733
Pou2f2
0.93228627328
Rxra
0.714333611668
Sp1
0.887451495053
Srf
0.913463596107
Tbp
0.650860624848
Tcf7l2
0.849462619421

### Feature optimization

In [ ]:
importances = my_model.get_fscore()
importance_frame = pd.DataFrame({'Importance': list(importances.values()), 'Feature': list(importances.keys())})
importance_frame.sort_values(by = 'Importance', inplace = True)
importance_frame.plot(kind = 'barh', x = 'Feature', figsize = (8,8), color = 'orange')

In [ ]:
importance_frame

In [ ]:
# select features using threshold
from sklearn.feature_selection import SelectFromModel
thresholds = np.sort(my_model.feature_importances_)
for thresh in thresholds:
	# select features using threshold
	selection = SelectFromModel(my_model, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# train model
	selection_model = XGBClassifier()
	selection_model.fit(select_X_train, y_train)
	# eval model
	select_X_test = selection.transform(X_test)
	y_pred = selection_model.predict(select_X_test)
	predictions = [round(value) for value in y_pred]
	accuracy = accuracy_score(y_test, predictions)
	print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))


## The next steps in predictions
1. Create an additional; feature, that uses the best PWM for prediction. This should would be used during the comparison stage, to show how this model performs compared with a pwm model. We can also compare it with the pure k-mer scoring. 
2. An additional thing to validate or argue in my thesis is the value of the k-mer scoring approach being used. I need to campare the few approaches I can come across on their own. This could later be moved to chapter three or something. 
3. If time allows, how hard would it be for me to include a DNAshape feature to my model as well?
4. When I have multiple features to work with, how can I choose te most predictive features? The number of features that would have optimal performance. 
    - Here is where recursive feature elimination may be of some use
5. What else?

In [ ]:
feature_frame.corr()

max using 2000 each for all the features
AUC 0.9342615
AUPRC 0.924861874448

max using a but the dnase feature with 4000
AUC 0.897419625
AUPRC 0.892977974024


max with all but two
AUC 0.914374375
AUPRC 0.90579930936

max for all after changing the scoing function to Energy score
AUC 0.941812875
AUPRC 0.93820425951


max
AUC 0.850881613639
AUPRC 0.834658458345

In [ ]:
for tf in tf_list:
    print tf
    print get_contigmers(tf)

In [ ]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1) 

In [ ]:
optimized_GBM.fit(feature_frame, y_train)

In [ ]:
optimized_GBM.grid_scores_

In [ ]:
cv_params = {'learning_rate': [0.1, 0.01], 'subsample': [0.7,0.8,0.9]}
ind_params = {'n_estimators': 1000, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth': 3, 'min_child_weight': 1}


optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1)
optimized_GBM.fit(feature_frame, y_train)

In [ ]:
GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'subsample': [0.7, 0.8, 0.9], 'learning_rate': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

[mean: 0.82907, std: 0.02203, params: {'max_depth': 3, 'min_child_weight': 1},
 mean: 0.82895, std: 0.02155, params: {'max_depth': 3, 'min_child_weight': 3},
 mean: 0.82891, std: 0.02206, params: {'max_depth': 3, 'min_child_weight': 5},
 mean: 0.82663, std: 0.02266, params: {'max_depth': 5, 'min_child_weight': 1},
 mean: 0.82706, std: 0.02249, params: {'max_depth': 5, 'min_child_weight': 3},
 mean: 0.82644, std: 0.02345, params: {'max_depth': 5, 'min_child_weight': 5},
 mean: 0.82056, std: 0.02175, params: {'max_depth': 7, 'min_child_weight': 1},
 mean: 0.81939, std: 0.02062, params: {'max_depth': 7, 'min_child_weight': 3},
 mean: 0.82089, std: 0.02104, params: {'max_depth': 7, 'min_child_weight': 5}]


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'subsample': [0.7, 0.8, 0.9], 'learning_rate': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

#learning rate

[mean: 0.82960, std: 0.02174, params: {'subsample': 0.7, 'learning_rate': 0.1},
 mean: 0.82907, std: 0.02203, params: {'subsample': 0.8, 'learning_rate': 0.1},
 mean: 0.82858, std: 0.02144, params: {'subsample': 0.9, 'learning_rate': 0.1},
 mean: 0.82084, std: 0.01890, params: {'subsample': 0.7, 'learning_rate': 0.01},
 mean: 0.82065, std: 0.01878, params: {'subsample': 0.8, 'learning_rate': 0.01},
 mean: 0.82001, std: 0.01869, params: {'subsample': 0.9, 'learning_rate': 0.01}]

In [ ]:
optimized_GBM.grid_scores_

In [ ]:
feature_frame.corr()

In [ ]:
xgdmat = xgb.DMatrix(feature_frame, y_train) # Create our DMatrix to make XGBoost more efficient

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

In [ ]:
cv_xgb.tail(5)

In [ ]:
#my_model = train_xgboost(feature_frame[["max_kmer_score","dn_hg_score2"]], y_train)

### Use the created model to predict in a new set of data

In [ ]:
#pssm = get_jaspar_pssm("MA0466.1")

In [ ]:
def prepare_learning_data(feature_frame, pos_size, neg_size):
    """
    Given a pandas dataframe with the features,
    """
    
    #import pandas as pd
    #import numpy as np
    #from sklearn.datasets import dump_svmlight_file
    
    y = np.concatenate((np.ones(pos_size), np.ones(neg_size)*-1), axis=0)

    target = pd.Series.from_array(y)
    target = target.apply(int)
    target = target.to_frame(name="Target")
    
    target_f1 = feature_frame.T.append(target.T).T


    cutoff = target_f1.count()[0]

    ids = pd.Series.from_array(np.arange(1, cutoff+1))
    ids = ids.apply(int)

    target_f1 = target_f1.T.append(ids.to_frame(name="Id").T).T


    X = target_f1[np.setdiff1d(target_f1.columns,['Id','Target'])]
    y = target_f1.Target
    
    return target_f1

These are very disappointing performaces. Why so? Is it a failure in the scorin functions, the k-mers or the model, of the choice of negative sequences? What would happen if we added more features? Which features would these be? So, what then is the next step:
- Expand my features to include PWM
- Use the maximum k-mer score rather than the sum
- test a different TFs
- make the pos and negative set equal
- Include counts in frequency difference directly, sclalled in the same way as E-scores
- Test other machine learning aprroaches like SVM
- Use a k-mer model from kmerSVM and use it to score the sequences

Write up on what we will have learned so far. Spend the morning implementing the above and the afternoon in the cafe, writing up on where I am so far, especially the introduction and metholology section. 

In [ ]:
# trim_to_p = min(len(pos_bed_p), len(neg_bed_p))
# #trim_to_p = 2000
# pos_bed_p = pos_bed_p.head(trim_to_p)
# neg_bed_p = neg_bed_p.head(trim_to_p)

# combined_bed_p = pos_bed_p.append(neg_bed_p, ignore_index=True)

# E_score_combined_p = get_kmer_score(combined_bed_p, sum_kmer_score, E_score_dict)
# max_score_combined_p = get_kmer_score(combined_bed_p, max_score_kmer, E_score_dict)
# dn_hg_score_combined2_p = get_kmer_score(combined_bed_p, max_score_kmer, dn_hg_dict2)
# dn_hg_score_combined_p = get_kmer_score(combined_bed_p, max_score_kmer, dn_hg_dict)
# dnase_scores_p = apply_get_max_dnase(combined_bed_p)

# feature_frame_p = pd.DataFrame(E_score_combined_p, columns=["kmer_score"])
# feature_frame_p ["max_kmer_score"] = max_score_combined_p
# feature_frame_p["dn_hg_score"] = dn_hg_score_combined_p
# feature_frame_p["dn_hg_score2"] = dn_hg_score_combined2_p
# feature_frame_p["dnase"] = dnase_scores_p